## Importing Libraries

In [120]:
import numpy as np
import pandas as pd
import glob
import zipfile

## Loading Data

#### Training Data

In [121]:
df_training = pd.read_csv("datasets_hi4/train-data.csv", sep=';')
df_training.drop_duplicates(inplace=True)
print("le shape de mon training dataframe is ", df_training.shape)

print(len(df_training['id_product'].unique()))

le shape de mon training dataframe is  (1762970, 19)
244857


In [122]:
df_training['Reference proxy'] = df_training['Reference proxy'].apply(lambda x: int(x.replace('reference-', '')))
df_training['Product  Line proxy'] = df_training['Product  Line proxy'].apply(lambda x: int(x.replace('Product Line-', '')))
df_training['Division proxy'] = df_training['Division proxy'].apply(lambda x: int(x.replace('Division-', '')))
df_training['Customer Persona proxy'] = df_training['Customer Persona proxy'].apply(lambda x: int(x.replace('Customer Segmentation-', '')))
df_training['Strategic Product Family proxy'] = df_training['Strategic Product Family proxy'].apply(lambda x: int(x.replace('Strategic Product Family-', '')))

In [123]:
df_training.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1762970 entries, 0 to 3173343
Data columns (total 19 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   index                           int64 
 1   id_product                      int64 
 2   Region                          object
 3   Country                         object
 4   Site                            object
 5   Operations                      object
 6   Zone                            object
 7   Cluster                         object
 8   Reference proxy                 int64 
 9   Product  Line proxy             int64 
 10  Division proxy                  int64 
 11  Customer Persona proxy          int64 
 12  Strategic Product Family proxy  int64 
 13  Product Life cycel status       object
 14  Date                            object
 15  Month 1                         object
 16  Month 2                         object
 17  Month 3                         object
 18  Month 4

In [124]:
df_training.isna().sum()

index                                   0
id_product                              0
Region                                  0
Country                                 0
Site                                    0
Operations                              0
Zone                                    0
Cluster                                 0
Reference proxy                         0
Product  Line proxy                     0
Division proxy                          0
Customer Persona proxy                  0
Strategic Product Family proxy          0
Product Life cycel status         1523673
Date                                    0
Month 1                            195865
Month 2                                 0
Month 3                                 0
Month 4                                 0
dtype: int64

In [125]:
df_training['Month 1'] = df_training['Month 1'].fillna(df_training['Month 2'])

In [126]:
df_training['Month 1'] = df_training['Month 1'].apply(lambda x: int(str(x).replace(' ', '')))
df_training['Month 2'] = df_training['Month 2'].apply(lambda x: int(str(x).replace(' ', '')))
df_training['Month 3'] = df_training['Month 3'].apply(lambda x: int(str(x).replace(' ', '')))
df_training['Month 4'] = df_training['Month 4'].apply(lambda x: int(str(x).replace(' ', '')))

In [127]:
df_training.isna().sum()

index                                   0
id_product                              0
Region                                  0
Country                                 0
Site                                    0
Operations                              0
Zone                                    0
Cluster                                 0
Reference proxy                         0
Product  Line proxy                     0
Division proxy                          0
Customer Persona proxy                  0
Strategic Product Family proxy          0
Product Life cycel status         1523673
Date                                    0
Month 1                                 0
Month 2                                 0
Month 3                                 0
Month 4                                 0
dtype: int64

In [128]:
### Adding a Year columns to the data set

country_mapping = {'DE': 'Germany', 'CN': 'China', 'GB': 'United Kingdom', 'AU': 'Australia', 'ES': 'Spain', 'NL': 'Netherlands', 'US': 'United States', 'DK': 'Denmark', 'BE': 'Belgium', 'FR': 'France', 'IT': 'Italy', 'PL': 'Poland', 'SE': 'Sweden', 'TW': 'Taiwan', 'JP': 'Japan', 'HK': 'Hong Kong', 'KR': 'South Korea', 'PH': 'Philippines', 'MY': 'Malaysia', 'SG': 'Singapore', 'TH': 'Thailand', 'Id': 'Indonesia', 'FJ': 'Fiji', 'VN': 'Vietnam', 'BN': 'Brunei', 'NZ': 'New Zealand', 'MM': 'Myanmar', 'KH': 'Cambodia', 'MN': 'Mongolia', 'LA': 'Laos', 'PG': 'Papua New Guinea'}
df_training['Country'] = df_training['Country'].map(country_mapping)
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

def get_Year(x) : 
    return x.split(' ')[1]

def get_MonthMin(x) : 
    return x.split(' ')[0].split('-')[0]

def get_MonthMax(x) : 
    return x.split(' ')[0].split('-')[1]

def get_Month2(x) :
    return months[months.index(x) + 1]


def get_Month3(x) :
    return months[months.index(x) - 1]

df_training['Year'] = df_training['Date'].apply(get_Year)



#df_training['Month_1'] = df_training['Date'].apply(get_MonthMin)
#df_training['Month_2'] = df_training['Month_1'].apply(get_Month2)
#df_training['Month_4'] = df_training['Date'].apply(get_MonthMax)
#df_training['Month_3'] = df_training['Month_4'].apply(get_Month3)
print(df_training.shape)
df_training.head()

(1762970, 20)


,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4,Year
0,645874,156160,EUROPE,Germany,NL_DC_Venray,Europe Operations,DACH,Germany,13523,4,3,3,12,NaN,may-aug 2021,0,0,0,0,2021
1,469488,224631,CHINA,China,CN_DC_Shanghai,China Operations,China & HK,China,12778,4,3,14,9,NaN,jan-apr 2021,0,1,0,0,2021
2,348904,104047,EUROPE,United Kingdom,NL_DC_HLD,Europe Operations,UK and Ireland,United Kingdom,3513,3,2,23,7,NaN,jan-apr 2021,0,2,0,0,2021
3,1725822,11823,EAJP,Australia,AU_DC_Perth,International Operations,Pacific,Australia,672,1,1,11,3,NaN,jan-apr 2023,0,0,0,0,2023
4,404781,159924,EUROPE,Spain,NL_DC_HLD,Europe Operations,Iberia,Spain,3496,3,2,14,7,NaN,jan-apr 2021,0,0,0,0,2021


### GSCPI Data

In [129]:
df_gscpi = pd.read_csv("datasets_hi4/extra-dataset/GSCPI_data.csv")
print("le shape de mon gscpi dataframe is ", df_gscpi.shape)
df_gscpi.head()

le shape de mon gscpi dataframe is  (43, 2)


,Year-Month,GSCPI
0,2020-01,0.090108
1,2020-02,1.191976
2,2020-03,2.546002
3,2020-04,3.161703
4,2020-05,2.537110


### LPI Extend Data

In [130]:
df_lpi = pd.read_csv("datasets_hi4/extra-dataset/LPIextend.csv")
print("le shape de mon gscpi dataframe is ", df_lpi.shape)
df_lpi.head()

le shape de mon gscpi dataframe is  (250, 26)


,Unnamed: 0,ID,Country,population (2023),area,landAreaKm,unMember,netChange,growthRate,worldPercentage,...,Infrastructure Score,Infrastructure Grouped Rank,International Shipments Score,International Shipments Grouped Rank,Logistics Competence and Quality Score,Logistics Competence and Quality Grouped Rank,Timeliness Score,Timeliness Grouped Rank,Tracking and Tracing Score,Tracking and Tracing Grouped Rank
0,0,0,Singapore,6014723.0,710.0,718.0,True,0.0012,0.0065,0.0008,...,4.6,1.0,4.0,2.0,4.4,1.0,4.3,1.0,4.4,1.0
1,1,1,Finland,5545475.0,338424.0,303940.0,True,0.0001,0.0009,0.0007,...,4.2,5.0,4.1,1.0,4.2,3.0,4.3,1.0,4.2,3.0
2,2,2,Denmark,5910913.0,43094.0,40000.0,True,0.0009,0.0049,0.0007,...,4.1,9.0,3.6,14.0,4.1,9.0,4.1,10.0,4.3,2.0
3,3,3,Germany,83294633.0,357114.0,349390.0,True,-0.0012,-0.0009,0.0104,...,4.3,3.0,3.7,8.0,4.2,3.0,4.1,10.0,4.2,3.0
4,4,4,Netherlands,17618299.0,41850.0,33670.0,True,0.0017,0.0031,0.0022,...,4.2,5.0,3.7,8.0,4.2,3.0,4.0,17.0,4.2,3.0


In [131]:
df_lpi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 26 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Unnamed: 0                                     250 non-null    int64  
 1   ID                                             250 non-null    int64  
 2   Country                                        250 non-null    object 
 3   population (2023)                              234 non-null    float64
 4   area                                           234 non-null    float64
 5   landAreaKm                                     234 non-null    float64
 6   unMember                                       234 non-null    object 
 7   netChange                                      226 non-null    float64
 8   growthRate                                     234 non-null    float64
 9   worldPercentage                                228 non

In [132]:
df_lpi.describe()

,Unnamed: 0,ID,population (2023),area,landAreaKm,netChange,growthRate,worldPercentage,density,densityMi,...,Infrastructure Score,Infrastructure Grouped Rank,International Shipments Score,International Shipments Grouped Rank,Logistics Competence and Quality Score,Logistics Competence and Quality Grouped Rank,Timeliness Score,Timeliness Grouped Rank,Tracking and Tracing Score,Tracking and Tracing Grouped Rank
count,250.000000,250.000000,2.340000e+02,2.340000e+02,2.340000e+02,226.000000,234.000000,228.000000,234.000000,234.000000,...,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000
mean,124.500000,124.500000,3.437442e+07,5.814500e+05,5.571123e+05,0.010306,0.009737,0.004407,451.288182,1168.836388,...,2.921583,66.856115,2.925180,66.561151,3.028777,67.194245,3.242446,67.143885,3.051079,67.079137
std,72.312977,72.312977,1.373864e+08,1.761841e+06,1.689972e+06,0.034774,0.012350,0.017375,1979.362419,5126.548664,...,0.720785,39.516381,0.523518,40.346886,0.645637,40.045966,0.565042,40.601557,0.675489,39.758157
min,0.000000,0.000000,5.180000e+02,4.400000e-01,4.400000e-01,-0.028600,-0.074500,0.000000,0.138000,0.357400,...,1.700000,1.000000,1.700000,1.000000,1.800000,1.000000,2.100000,1.000000,1.600000,1.000000
25%,62.250000,62.250000,4.225982e+05,2.650000e+03,2.625875e+03,0.000000,0.002325,0.000100,39.747650,102.946450,...,2.400000,30.000000,2.500000,26.000000,2.500000,33.000000,2.800000,35.000000,2.450000,34.000000
50%,124.500000,124.500000,5.643895e+06,8.119950e+04,7.568925e+04,0.000900,0.008200,0.000750,97.481000,252.475800,...,2.700000,68.000000,2.900000,68.000000,2.900000,65.000000,3.200000,65.000000,3.000000,65.000000
75%,186.750000,186.750000,2.324537e+07,4.304258e+05,4.047876e+05,0.008000,0.016850,0.002925,242.928650,629.185350,...,3.600000,89.000000,3.400000,102.000000,3.600000,103.000000,3.600000,102.000000,3.600000,101.500000
max,249.000000,249.000000,1.428628e+09,1.709824e+07,1.637687e+07,0.418400,0.049800,0.178500,21402.705200,55433.006400,...,4.600000,139.000000,4.100000,140.000000,4.400000,140.000000,4.300000,139.000000,4.400000,140.000000


### World Bank Economy Data

In [133]:
df_worldEconomy = pd.read_csv("datasets_hi4/extra-dataset/worldbank_economic_data.csv")
df_worldEconomy['Year'] = df_worldEconomy['Year'].astype(str)
print("le shape de mon gscpi dataframe is ", df_worldEconomy.shape)
df_worldEconomy.head()

le shape de mon gscpi dataframe is  (1064, 11)


,Country,Year,"Agriculture, forestry, and fishing, value added (annual % growth)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),GDP (current US$),Gross capital formation (annual % growth),Imports of goods and services (annual % growth),"Industry (including construction), value added (annual % growth)","Manufacturing, value added (annual % growth)","Services, value added (annual % growth)"
0,Afghanistan,2020,5.94,NaN,NaN,2.014345e+10,NaN,NaN,-4.63,-3.54,-5.92
1,Afghanistan,2021,-2.83,NaN,NaN,1.458314e+10,NaN,NaN,-14.19,-7.22,-32.74
2,Afghanistan,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Africa Eastern and Southern,2020,3.15,-11.41,-1.93,9.275933e+11,-8.37,-11.94,-4.86,-7.06,-3.05


In [134]:
df_worldEconomy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1064 entries, 0 to 1063
Data columns (total 11 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Country                                                            1064 non-null   object 
 1   Year                                                               1064 non-null   object 
 2   Agriculture, forestry, and fishing, value added (annual % growth)  661 non-null    float64
 3   Exports of goods and services (annual % growth)                    541 non-null    float64
 4   Final consumption expenditure (annual % growth)                    561 non-null    float64
 5   GDP (current US$)                                                  741 non-null    float64
 6   Gross capital formation (annual % growth)                          527 non-null    float64
 7   Imports of goods and ser

In [135]:
df_worldEconomy.describe()

,"Agriculture, forestry, and fishing, value added (annual % growth)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),GDP (current US$),Gross capital formation (annual % growth),Imports of goods and services (annual % growth),"Industry (including construction), value added (annual % growth)","Manufacturing, value added (annual % growth)","Services, value added (annual % growth)"
count,661.000000,541.000000,561.000000,7.410000e+02,527.000000,545.000000,660.000000,580.000000,658.000000
mean,1.491997,1.833641,2.263298,3.169917e+12,3.299715,3.315174,1.634636,1.145000,1.416824
std,8.111301,22.285346,6.055425,1.050376e+13,41.152589,16.372478,11.198461,8.870702,7.489096
min,-32.360000,-74.420000,-27.510000,5.174657e+07,-277.920000,-61.940000,-43.150000,-55.170000,-56.060000
25%,-1.010000,-9.160000,-1.360000,1.181391e+10,-6.350000,-8.150000,-3.027500,-3.275000,-2.560000
50%,2.040000,4.300000,3.140000,6.838084e+10,2.830000,5.460000,1.930000,2.460000,3.040000
75%,3.940000,11.000000,5.520000,7.864600e+11,10.045000,13.410000,5.310000,6.302500,5.820000
max,59.880000,207.460000,24.670000,1.005620e+14,740.260000,113.810000,162.670000,23.890000,46.840000


### World Bank Inflation

In [136]:
df_worldInfla = pd.read_csv("datasets_hi4/extra-dataset/worldbank_inflation_data.csv")
print("le shape de mon gscpi dataframe is ", df_worldInfla.shape)

df_worldInfla = pd.merge(df_worldInfla, df_gscpi, on='Year-Month', how='left')

df_worldInfla.head()

le shape de mon gscpi dataframe is  (8127, 4)


,Country,Year-Month,Energy Price Index,Headline Consumer Price Index,GSCPI
0,Afghanistan,2020-01,NaN,154.191402,0.090108
1,Afghanistan,2020-02,NaN,153.366469,1.191976
2,Afghanistan,2020-03,NaN,153.676014,2.546002
3,Afghanistan,2020-04,NaN,NaN,3.161703
4,Afghanistan,2020-05,NaN,NaN,2.537110


In [137]:
def get_Date(x) : 
    year = x.split('-')[0]
    month = int(x.split('-')[1])

    if year == '2023' :
        if month > 4 : 
            return 'may-jul 2023'
        else : 
            return 'jan-apr 2023'
    
    if month <= 4 : 
        return 'jan-apr ' + year
    elif month > 4 and month <=8 : 
        return 'may-aug ' + year
    else : 
        return 'sep-dec ' + year

df_worldInfla['Date'] = df_worldInfla['Year-Month'].apply(get_Date)

result_df1 = df_worldInfla.groupby(['Country', 'Date'])['Energy Price Index'].apply(lambda x: x.tolist()).reset_index(name='Energy_Price_Index_List')
result_df2 = df_worldInfla.groupby(['Country', 'Date'])['Headline Consumer Price Index'].apply(lambda x: x.tolist()).reset_index(name='Headline_Consumer_Index_List')
result_df3 = df_worldInfla.groupby(['Country', 'Date'])['GSCPI'].apply(lambda x: x.tolist()).reset_index(name='GSCPI_List')

print(result_df1.shape)
print(result_df2.shape)
print(result_df3.shape)

result_df1_expanded = result_df1['Energy_Price_Index_List'].apply(pd.Series)
result_df1_expanded = result_df1_expanded.rename(columns=lambda x: f'Energy_Price_Index_{x + 1}')

result_df2_expanded = result_df2['Headline_Consumer_Index_List'].apply(pd.Series)
result_df2_expanded = result_df2_expanded.rename(columns=lambda x: f'Headline_Consumer_Index_{x + 1}')

result_df3_expanded = result_df3['GSCPI_List'].apply(pd.Series)
result_df3_expanded = result_df3_expanded.rename(columns=lambda x: f'GSCPI_{x + 1}')

df_final1 = pd.concat([result_df1, result_df1_expanded], axis=1)
df_final2 = pd.concat([result_df2, result_df2_expanded], axis=1)
df_final3 = pd.concat([result_df3, result_df3_expanded], axis=1)

df_final = pd.merge(df_final1, df_final2, on = ['Country', 'Date'], how='inner')
df_final = pd.merge(df_final, df_final3, on = ['Country', 'Date'], how='inner')

df_final = df_final.drop(['Headline_Consumer_Index_List', 'Energy_Price_Index_List', 'GSCPI_List'], axis=1)

print(df_final.shape)
df_final.head(14)

(2079, 3)
(2079, 3)
(2079, 3)
(2079, 14)


,Country,Date,Energy_Price_Index_1,Energy_Price_Index_2,Energy_Price_Index_3,Energy_Price_Index_4,Headline_Consumer_Index_1,Headline_Consumer_Index_2,Headline_Consumer_Index_3,Headline_Consumer_Index_4,GSCPI_1,GSCPI_2,GSCPI_3,GSCPI_4
0,Afghanistan,jan-apr 2020,NaN,NaN,NaN,NaN,154.191402,153.366469,153.676014,NaN,0.090108,1.191976,2.546002,3.161703
1,Afghanistan,jan-apr 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.329892,1.908749,2.199072,2.682921
2,Afghanistan,jan-apr 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.567542,2.732736,2.750733,3.417682
3,Afghanistan,jan-apr 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.023994,-0.264112,-1.153952,-1.327040
4,Afghanistan,may-aug 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.537110,2.252427,2.742415,1.354547
5,Afghanistan,may-aug 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.981772,2.694348,2.922301,3.232759
6,Afghanistan,may-aug 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.637340,2.341098,1.771378,1.459910
7,Afghanistan,may-jul 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.544344,-1.085352,-0.831945,NaN
8,Afghanistan,sep-dec 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.615690,0.121514,0.740285,1.661260
9,Afghanistan,sep-dec 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.275933,3.808737,4.212181,4.293310


In [138]:
#merge training + worldinfla + gscpi

df = pd.merge(df_training, df_final, on=['Country', 'Date'], how='left')
print(df.shape)
df.head()

(1762970, 32)


,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,...,Energy_Price_Index_3,Energy_Price_Index_4,Headline_Consumer_Index_1,Headline_Consumer_Index_2,Headline_Consumer_Index_3,Headline_Consumer_Index_4,GSCPI_1,GSCPI_2,GSCPI_3,GSCPI_4
0,645874,156160,EUROPE,Germany,NL_DC_Venray,Europe Operations,DACH,Germany,13523,4,...,109.501,110.149,108.170000,108.486000,109.013000,109.118000,2.981772,2.694348,2.922301,3.232759
1,469488,224631,CHINA,China,CN_DC_Shanghai,China Operations,China & HK,China,12778,4,...,100.900,100.900,129.344851,130.113998,129.473043,129.088469,1.329892,1.908749,2.199072,2.682921
2,348904,104047,EUROPE,United Kingdom,NL_DC_HLD,Europe Operations,UK and Ireland,United Kingdom,3513,3,...,106.500,112.000,109.300000,109.400000,109.700000,110.400000,1.329892,1.908749,2.199072,2.682921
3,1725822,11823,EAJP,Australia,AU_DC_Perth,International Operations,Pacific,Australia,672,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,404781,159924,EUROPE,Spain,NL_DC_HLD,Europe Operations,Iberia,Spain,3496,3,...,102.303,106.463,104.500000,103.900000,104.900000,106.100000,1.329892,1.908749,2.199072,2.682921


In [139]:
# + world bank
df = pd.merge(df, df_worldEconomy, on=['Country', 'Year'], how='left')
df.shape

(1762970, 41)

In [140]:
# + lpi
df = pd.merge(df, df_lpi, on=['Country'], how='left')
df.shape

(1762970, 66)

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1762970 entries, 0 to 1762969
Data columns (total 66 columns):
 #   Column                                                             Dtype  
---  ------                                                             -----  
 0   index                                                              int64  
 1   id_product                                                         int64  
 2   Region                                                             object 
 3   Country                                                            object 
 4   Site                                                               object 
 5   Operations                                                         object 
 6   Zone                                                               object 
 7   Cluster                                                            object 
 8   Reference proxy                                                    int64  
 9   Pr

In [142]:
len(df['Country'].unique())

31

In [143]:
df['unMember'] = df['unMember'].astype(int) 

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1762970 entries, 0 to 1762969
Data columns (total 66 columns):
 #   Column                                                             Dtype  
---  ------                                                             -----  
 0   index                                                              int64  
 1   id_product                                                         int64  
 2   Region                                                             object 
 3   Country                                                            object 
 4   Site                                                               object 
 5   Operations                                                         object 
 6   Zone                                                               object 
 7   Cluster                                                            object 
 8   Reference proxy                                                    int64  
 9   Pr

In [145]:
from sklearn.preprocessing import LabelEncoder

string_columns = df.select_dtypes(include=['object']).columns

# Appliquer LabelEncoder à chaque colonne de type chaîne
for col in string_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [146]:
nan_counts = df.isna().sum()
nan_counts.to_csv('nan_counts.csv', index=True)

In [147]:
df = df.fillna(df.mean())

In [148]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.2, random_state=1337)

y_training = train['Month 4']
y_valid = valid['Month 4']


train = train.drop(['Month 4'], axis=1)
valid = valid.drop(['Month 4'], axis=1)


print(train.shape)
print(valid.shape)
train.head()
print(y_valid.shape)


(1410376, 65)
(352594, 65)
(352594,)


In [149]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Créer l'instance du modèle XGBoost
model = xgb.XGBRegressor(objective ='reg:squarederror')

# Entraîner le modèle
model.fit(train, y_training)

# Faire des prédictions
y_pred = model.predict(valid)

r2 = r2_score(y_valid, y_pred)
# Évaluer le modèle
mse = mean_squared_error(y_valid, y_pred)

print("r2 Score ", r2)
print(f"Erreur quadratique moyenne : {mse}")

# Utiliser le modèle pour prédire de nouvelles valeurs
# new_data est un nouveau DataFrame contenant les caractéristiques de nouvelles observations
# predictions = model.predict(new_data)

r2 Score  0.38782159161410545
Erreur quadratique moyenne : 244005.1910455105


In [97]:
print(y_valid.mean())

26.1457965819044


In [98]:
print(y_pred.mean())

26.637924


In [101]:
print(y_pred[:100])

[1.1772617e+00 1.9555314e+00 7.1999782e-01 7.1999782e-01 8.6247146e-01
 8.6247146e-01 1.3197351e+00 9.6166492e-01 1.1772617e+00 1.8423846e+00
 5.8362526e-01 8.6247146e-01 2.7947366e+00 8.6139421e+00 1.6550417e+01
 7.1999782e-01 1.9342205e+01 8.6247146e-01 8.6247146e-01 7.1999782e-01
 7.6806198e+01 8.6247146e-01 3.6192766e-01 8.1919169e-01 3.6192766e-01
 8.1919169e-01 9.6166492e-01 9.6166492e-01 8.6247146e-01 6.8281901e-01
 3.6217032e+02 5.7045546e+00 3.6192766e-01 7.1999782e-01 9.6166492e-01
 5.6065960e+01 6.4389768e+00 9.6166492e-01 1.3197351e+00 9.6166492e-01
 1.2023059e+02 8.6247146e-01 1.7216524e+00 1.6443672e+00 9.9963903e-01
 6.8281901e-01 9.6166492e-01 3.6192766e-01 3.6192766e-01 7.4730444e-01
 1.8406282e+01 8.6247146e-01 7.1999782e-01 3.6192766e-01 8.6247146e-01
 1.1772617e+00 1.1772617e+00 1.5060557e+00 5.3023979e+01 1.6666566e+00
 3.6192766e-01 3.6215630e+01 7.1999782e-01 2.0699480e+01 7.8284411e+00
 5.8468018e+00 1.1772617e+00 8.6247146e-01 1.8423846e+00 1.3197351e+00
 3.892